In [1]:
import tensorflow as tf
import sys
import matplotlib.pyplot as plt
import numpy as np
import os

In [2]:
sys.path.append("../");

In [3]:
import sropts
from neural_networks import *

In [4]:
# Download / load MNIST data
train_dir = '../data/MNIST/';
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(train_dir, one_hot=True);

Extracting ../data/MNIST/train-images-idx3-ubyte.gz
Extracting ../data/MNIST/train-labels-idx1-ubyte.gz
Extracting ../data/MNIST/t10k-images-idx3-ubyte.gz
Extracting ../data/MNIST/t10k-labels-idx1-ubyte.gz


In [5]:
# Parms
batch_size = 64;
image_size = 28;
category = 10;
learning_rate=1e-3;
chkpt_dir = "../chkpt/";
if not os.path.exists(chkpt_dir):
    os.makedirs(chkpt_dir);

In [6]:
# Build network

m_nn = NeuralNetworks.ConvMNIST(64);
x = tf.placeholder(dtype=tf.float32,shape=(batch_size, image_size, image_size,1), name='in-img');
y = tf.placeholder(dtype=tf.float32, shape=(batch_size, category), name='in-label');
predict = m_nn.nn_predict(x); 
m_loss = m_nn.loss(predict=predict, real=y);
correct_prediction = tf.equal(tf.argmax(predict,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

opt_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(m_loss);
init_op = tf.global_variables_initializer();

saver = tf.train.Saver();


In [9]:
#Training phase
with tf.Session() as sess:
    sess.run(init_op);
    for epoch in xrange(20):
        count = 0;
        while count <= mnist.train.num_examples-batch_size:
            count += batch_size;
            [input_x, input_y] = mnist.train.next_batch(batch_size=batch_size);
            input_x = np.reshape(input_x, newshape=(batch_size,28,28,1));
            
            [comp_loss,comp_acc, __] = sess.run([m_loss, accuracy, opt_op], feed_dict={
                    x: input_x,
                    y: input_y
                });
            
            print "\r\b epoch:",epoch, " image#:", count, " loss:", comp_loss, "acc:", comp_acc,
        saver.save(sess, chkpt_dir+'model.ckpt', global_step=(epoch+1)*count)
        print ""

 epoch: 0  image#: 54976  loss: 0.00841743 acc: 1.0 
 epoch: 1  image#: 54976  loss: 0.0190453 acc: 0.984375 
 epoch: 2  image#: 54976  loss: 0.04944 acc: 0.984375 
 epoch: 3  image#: 54976  loss: 0.0293756 acc: 0.984375 
 epoch: 4  image#: 54976  loss: 0.0282042 acc: 0.984375 
 epoch: 5  image#: 54976  loss: 0.0479051 acc: 0.984375 
 epoch: 6  image#: 54976  loss: 0.0239702 acc: 0.984375 
 epoch: 7  image#: 54976  loss: 0.00479308 acc: 1.0 
 epoch: 8  image#: 54976  loss: 0.00339093 acc: 1.0 
 epoch: 9  image#: 54976  loss: 0.0848073 acc: 0.984375 
 epoch: 10  image#: 54976  loss: 0.00266584 acc: 1.0 
 epoch: 11  image#: 54976  loss: 0.000393906 acc: 1.0 
 epoch: 12  image#: 54976  loss: 0.00572929 acc: 1.0 
 epoch: 13  image#: 54976  loss: 0.000453032 acc: 1.0 
 epoch: 14  image#: 54976  loss: 0.063358 acc: 0.984375 
 epoch: 15  image#: 54976  loss: 2.38039e-06 acc: 1.0 
 epoch: 16  image#: 54976  loss: 8.73053e-05 acc: 1.0 
 epoch: 17  image#: 54976  loss: 1.60555e

In [11]:
# Testing phase
with tf.Session() as sess:
    saver.restore(sess, chkpt_dir+"model.ckpt-549120");
    count = 0;
    all_loss = [];
    all_acc = [];
    while count <= mnist.test.num_examples-batch_size:
        count += batch_size;
        [input_x, input_y] = mnist.test.next_batch(batch_size=batch_size);
        input_x = np.reshape(input_x, newshape=(batch_size,28,28,1));

        [comp_loss,comp_acc, __] = sess.run([m_loss, accuracy, opt_op], feed_dict={
                x: input_x,
                y: input_y
            });         
        all_loss.append(comp_loss);
        all_acc.append(comp_acc);
        print "\r\b loss:",comp_loss, "acc:", comp_acc,
    print "";
    print "averge loss:", np.mean(all_loss), "average acc:", np.mean(all_acc);

 loss: 0.0636326 acc: 0.984375 
averge loss: 0.0622173 average acc: 0.98137


In [ ]:
averge loss: 0.0345858 average acc: 0.992188

9984

In [17]:
mnist.test.num_examples

10000